<img src="https://aiisc.ai/defactify/img/factify_logo.png" width=300 align="left">
<center>
<h4>Baseline</h4>
<p>Factify is a Multi-Modal Fact Verification dataset released for a shared task as part of the <a href="https://aiisc.ai/defactify/">De-Factify workshop</a> in AAAI-21.</p>
</center>

In [ ]:
import os
import io
import requests
from PIL import Image
import pandas as pd
import zipfile

# Installation


Download the training data from the codalab competition page from the Participate > Files > Public Data
or use the drive library below

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1ig7XEYU1UKDHrHgDYgqiARWvNdswgFEX',
                                    dest_path='/content/public_folder.zip')

In [ ]:
#fill password
pswd=""

In [ ]:
with zipfile.ZipFile("public_folder.zip") as file:
  file.extractall(pwd = bytes(pswd, 'utf-8'))

In [ ]:
download_path = "public_folder/"

In [ ]:
train_df = pd.read_csv(download_path + "train.csv", index_col="Id")
train_df.head()

In [ ]:
train_df=train_df.head()

You can save images as you like, but we provide a template to store claim and support images seperately in directories created class-wise.

In [ ]:
image_directory = "images"
if not os.path.isdir(image_directory):
  os.makedirs(image_directory)
for i in ["claim", "document"]:
  if not os.path.isdir(image_directory + "/" + i):
    os.makedirs(image_directory + "/" + i)
  for cls in ["Support_Multimodal", "Support_Text", "Insufficient_Multimodal", "Insufficient_Text", "Refute"]:
    if not os.path.isdir(image_directory + "/" + i + "/" + cls):
      os.makedirs(image_directory + "/" + i + "/" + cls)

In [ ]:
for n, row in train_df.iterrows():
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
  response = requests.get(row["claim_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory + "/claim/" + row["Category"] + "/" + str(n) + ".jpg")

  response = requests.get(row["document_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory + "/document/" + row["Category"] + "/" + str(n) + ".jpg")

# Model

In [ ]:
# Model definition and training
!pip install sentence-transformers
!pip install keras_applications

In [ ]:
import nltk
import numpy as np
import cv2
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from resnet50 import ResNet50
import tensorflow
from keras.layers import Input
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
def get_feature_vector_fromPIL(img):
    feature_vector = feature_model.predict(img)
    a, b, c, n = feature_vector.shape
    feature_vector= feature_vector.reshape(b,n)
    return feature_vector

def calculate_similarity_cosine(vector1, vector2):
    #return 1 - distance.cosine(vector1, vector2)
    return cosine_similarity(vector1, vector2)

# This distance can be in range of [0,∞]. And this distance is converted to a [0,1]
def calculate_similarity_euclidean(vector1, vector2):
    return 1/(1 + np.linalg.norm(vector1- vector2))  

#Use ResNet-50 model as an image feature extractor
image_input = Input(shape=(224, 224, 3))
feature_model = ResNet50(input_tensor=image_input, include_top=False,weights='imagenet')

In [ ]:
def cosinesimilarity_Resnet50(a1,a2):
    x = image.load_img(a1, target_size=(224, 224))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x2 = image.load_img(a2, target_size=(224, 224))
    x2 = image.img_to_array(x2)
    x2 = np.expand_dims(x2, axis=0)
    x2 = preprocess_input(x2)
    image_similarity_cosine = calculate_similarity_cosine(get_feature_vector_fromPIL(x), get_feature_vector_fromPIL(x2))[0][0]
    return image_similarity_cosine

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
em1 = sbert_model.encode(list(train_df['claim']))
em2 = sbert_model.encode(list(train_df['document']))
ps = PorterStemmer()
sw = list(stopwords.words('english'))
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

In [ ]:
text_sim = []
img_sim = []
for i in tqdm(range(len(em1))):
    sim = cosine(em1[i], em2[i])
    text_sim.append(sim)
    
    sim2 = cosinesimilarity_Resnet50(
        'images/claim/' + train_df["Category"].values[i] + "/" + str(train_df.index.values[i]) + ".jpg",
        'images/document/' + train_df["Category"].values[i] + "/" + str(train_df.index.values[i]) + ".jpg",
        )
    img_sim.append(sim2)

train_df['Text_Sim'] = text_sim
train_df['Img_Sim'] = img_sim

In [ ]:
train_df.head()

# Validation

In [ ]:
val_df = pd.read_csv(download_path + "val.csv", index_col="Id")
val_df.head()

In [ ]:
image_directory_val = "images_val"
if not os.path.isdir(image_directory_val):
  os.makedirs(image_directory_val)
for i in ["claim", "document"]:
  if not os.path.isdir(image_directory_val + "/" + i):
    os.makedirs(image_directory_val + "/" + i)

In [ ]:
for n, row in val_df.iterrows():
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
  response = requests.get(row["claim_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory_val + "/claim/" + str(n) + ".jpg")

  response = requests.get(row["document_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory_val + "/document/" + str(n) + ".jpg")

In [ ]:
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
em1 = sbert_model.encode(list(val_df['claim']))
em2 = sbert_model.encode(list(val_df['document']))
ps = PorterStemmer()
sw = list(stopwords.words('english'))
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

In [ ]:
# Validation data prediction using trained model
text_sim = []
img_sim = []
for i in tqdm(range(len(em1))):
    sim = cosine(em1[i], em2[i])
    text_sim.append(sim)
    
    sim2 = cosinesimilarity_Resnet50(
        'images_val/claim/' + str(val_df.index.values[i]) + ".jpg",
        'images_val/document/' + str(val_df.index.values[i]) + ".jpg",
        )
    img_sim.append(sim2)

val_df['Text_Sim'] = text_sim
val_df['Img_Sim'] = img_sim

## Text-only Model

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
X_train = train_df[["Text_Sim"]]
y_train = train_df["Category"]

X_test = val_df[["Text_Sim"]]
y_test = val_df["Category"]

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
predictions = neigh.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = RandomForestClassifier(n_estimators=500,max_depth=10, random_state=16)
clf = clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

## Multimodal

In [ ]:
X_train = train_df[["Text_Sim", "Img_Sim"]]
y_train = train_df["Category"]

X_test = val_df[["Text_Sim", "Img_Sim"]]
y_test = val_df["Category"]

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
predictions = neigh.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

In [ ]:
clf = RandomForestClassifier(n_estimators=500,max_depth=10, random_state=16)
clf = clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
score = f1_score(y_test, predictions, average='weighted')
print(score)

## Test 

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1dAlXZVDc09gmaEwPHHw8H2BCBYtDH4gz',
                                    dest_path='/content/test.zip')

In [ ]:
#fill password
pswd=""

In [ ]:
with zipfile.ZipFile("test.zip") as file:
  file.extractall(pwd = bytes(pswd, 'utf-8'))

In [ ]:
download_path = ""

In [ ]:
test_df = pd.read_csv(download_path + "test.csv", index_col="Id")
test_df.head()

In [ ]:
image_directory_test = "images_test"
if not os.path.isdir(image_directory_test):
  os.makedirs(image_directory_test)
for i in ["claim", "document"]:
  if not os.path.isdir(image_directory_test + "/" + i):
    os.makedirs(image_directory_test + "/" + i)

In [ ]:
for n, row in test_df.iterrows():
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
  response = requests.get(row["claim_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory_test + "/claim/" + str(n) + ".jpg")

  response = requests.get(row["document_image"], headers=headers)
  img = Image.open(io.BytesIO(response.content))
  img = img.convert('RGB')
  img.save(image_directory_test + "/document/" + str(n) + ".jpg")

In [ ]:
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
em1 = sbert_model.encode(list(test_df['claim']))
em2 = sbert_model.encode(list(test_df['document']))
ps = PorterStemmer()
sw = list(stopwords.words('english'))
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

In [ ]:
# Test data prediction using trained model
text_sim = []
img_sim = []
for i in tqdm(range(len(em1))):
    sim = cosine(em1[i], em2[i])
    text_sim.append(sim)
    
    sim2 = cosinesimilarity_Resnet50(
        'images_test/claim/' + str(test_df.index.values[i]) + ".jpg",
        'images_test/document/' + str(test_df.index.values[i]) + ".jpg",
        )
    img_sim.append(sim2)

test_df['Text_Sim'] = text_sim
test_df['Img_Sim'] = img_sim

In [ ]:
# Multimodal Model

X_train = train_df[["Text_Sim", "Img_Sim"]]
y_train = train_df["Category"]

X_test = test_df[["Text_Sim", "Img_Sim"]]

In [ ]:
clf = RandomForestClassifier(n_estimators=500,max_depth=10, random_state=16)
clf = clf.fit(X_train,y_train)
predictions = clf.predict(X_test)

In [ ]:
res = {"Id": test_df.index, "Category": predictions}
answer = pd.DataFrame(res)

In [ ]:
answer.to_csv("answer.csv",index=False)
!zip answer.zip answer.csv